# openEO Platform - Sentinel-2 Suspended Particulate Matter Data Example
#### Sentinel-2 Suspended Particulate Matter

Currently, openEO Platform is only open for Early Adopters or within a free 30 day trial period.

To express your interest in becoming an Early Adopter you need to follow 2 steps:

1 - Connect an existing account to EGI check-in 2 - Apply to the openEO Platform virtual organization

The 2 steps are described in detail here: https://docs.openeo.cloud/join/early_adopter.html#preamble-registration-and-login-authentication

This notebook is based on the openEOPlatform github, sample-notebooks, here: https://github.com/openEOPlatform/sample-notebooks/blob/main/openEO%20Platform%20-%20Optical%20ARD.ipynb

### Import Libraries

Note: eo_utils.py must be in the same folder as the notebook. It can be retreived from the openEO github, here:
https://github.com/openEOPlatform/sample-notebooks

In [37]:
import openeo
import time
from datetime import datetime
import csv
from csv import writer
from eo_utils import *
from netCDF4 import Dataset
import numpy as np

In [38]:
openeo.__version__

'0.10.1a1'

### 1. Open the connection with openEO Platform. You can autenticate using EGI.

Please refer to the openEO Platform official documentation for details about the installation and authentication: https://docs.openeo.cloud/getting-started/python/

In [39]:
backend = "https://openeo.vito.be"
conn = openeo.connect(backend)
conn.list_collections()
conn = openeo.connect(backend).authenticate_oidc(provider_id='egi')

Authenticated using refresh token.


In [40]:
backend = "https://openeo.vito.be"
conn = openeo.connect(backend).authenticate_oidc(provider_id="egi")

Authenticated using refresh token.


### 2. Area of Interest selection

Defining Area of Interest (AOI) per centre coordinates and zoom level.
If necessary, draw and adjust the bounding box (bbox) over the map and get its coordinates with the getBbox command.

In [41]:
center = [32.72, -17.19]
zoom = 12

eoMap = openeoMap(center,zoom)
eoMap.map

Map(center=[32.72, -17.19], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [42]:
bbox = eoMap.getBbox()
print('west',bbox[0],'\neast',bbox[2],'\nsouth',bbox[1],'\nnorth',bbox[3])

west -17.327156066894535 
east -17.05284118652344 
south 32.66221132308069 
north 32.77774932798732


### 3. Define range of interest in time and requested bands from AGERA5

In [43]:
conn.describe_collection('TERRASCOPE_S2_SPM_V1')

{'assets': {'metadata_iso_19139': {'href': 'https://services.terrascope.be/collectioncatalogue/srv/api/records/urn:eop:VITO:TERRASCOPE_S2_SPM_V1/formatters/xml?approved=true',
   'roles': ['metadata', 'iso-19139'],
   'title': 'ISO 19139 metadata',
   'type': 'application/vnd.iso.19139+xml'},
  'thumbnail': {'href': 'https://services.terrascope.be/collectioncatalogue/images/quicklooks/TERRASCOPE_S2_SPM_V1',
   'roles': ['thumbnail'],
   'title': 'Thumbnail',
   'type': 'image/jpeg'}},
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['SPM', 'PIXELCLASSIFICATION_10M']},
  't': {'extent': ['2015-07-06T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-56, 83],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'}},
 'description': 'The SENTINEL-2 SPM corresponds to the Suspended Particulate Matter water quality products, units are expressed in (mg L-1).',
 'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
  'temporal': {'interval': [['2015-07-06T00:00:00Z', None]]}},
 'id': 'TERRASCOPE_S2_SPM_V1',
 'keywords': ['VITO',
  'TERRASCOPE',
  'COPERNICUS',
  'ESA',
  'Orthoimagery',
  'SENTINEL-2',
  'MSI',
  'Level-3',
  'Radiometry',
  'Water Quality',
  'Total Suspended Matter',
  'Suspended Particulate Matter'],
 'license': 'proprietary',
 'links': [{'href': 'https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice',
   'rel': 'license'},
  {'href': 'https://services.terrascope.be/catalogue/collections?uid=urn:eop:VITO:TERRASCOPE_S2_SPM_V1',
   'rel': 'alternate'},
  {'href': 'https://viewer.terrascope.be/?language=nl&bbox=0.9339867511378304,50.14520994379217,6.207424251137831,51.77292299745045&overlay=false&bgLayer=MapBox&date=2019-02-25T03:17:15.000Z&layer=TERRASCOPE_S2_SPM_V1',
   'rel': 'alternate'},
  {'href': 'https://services.terrascope.be/wmts/v2',
   'rel': 'wmts',
   'wmts:layer': 'TERRASCOPE_S2_SPM_V1'}],
 'providers': [{'name': 'VITO, on behalf of the Belgian Science Policy Office (BELSPO).',
   'roles': ['producer', 'licensor'],
   'url': 'https://terrascope.be'}],
 'stac_extensions': ['datacube'],
 'stac_version': '0.9.0',
 's

In [44]:
collection      = 'TERRASCOPE_S2_SPM_V1'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':'EPSG:4326'}
temporal_extent = ["2017-09-01", "2022-03-01"]
bands           = ['SPM','PIXELCLASSIFICATION_10M']


### 4. Save process as final step, we use NetCDF as output format

In [45]:
spm = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent)

spm_nc = spm.save_result(format="NETCDF")
print(spm_nc)

DataCube(<PGNode 'save_result' at 0x7f9b0024cc50>)


 ### 5. Create a batch job and start it. More info about batch jobs can be found here:

https://docs.openeo.cloud/getting-started/python/#batch-job-execution

https://open-eo.github.io/openeo-python-client/batch_jobs.html

In [46]:
job = conn.create_job(spm_nc)
job_id = job.job_id
if job_id:
    print("Batch job created with id: ",job_id)
    job.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  0f0075a2-ea3a-468c-a567-f7d819e19e6f


### 8. Get a description of the job and check its status.

In [47]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")

my_job = conn.job(job_id)
job_description = my_job.describe_job()

header = ['job_description', 'job_timestamp', 'job_id','job_status']
data = ['marismar_s2l3_spm_2017_2022_timeseries',timestampStr, job_id,job_description['status']]

with open('job_id.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerow(data)
    

    
print("Batch job with id: ",job_id, ' is ',job_description['status'])

Batch job with id:  0f0075a2-ea3a-468c-a567-f7d819e19e6f  is  queued


In [51]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")

my_job = conn.job(job_id)
job_description = my_job.describe_job()

header = ['job_description', 'job_timestamp', 'job_id','job_status']
data = ['marismar_s2l3_spm_2017_2022_timeseries',timestampStr, job_id,job_description['status']]

with open('job_id.csv', 'a') as f_object:

    writer_object = csv.writer(f_object)
  
    writer_object.writerow(data)
  
    #Close the file object
    f_object.close()
    
print("Batch job with id: ",job_id, ' is ',job_description['status'])

Batch job with id:  0f0075a2-ea3a-468c-a567-f7d819e19e6f  is  error


### 9. Download and visualize the results

In [ ]:
%%time

results = my_job.get_results()
metadata = results.get_metadata()
results.download_file("./marismar_s2l3_spm_2017_2021_timeseries.nc")

In [40]:
#download started at 20220523 13:39